In [43]:
import pandas as pd
import numpy as np

In [109]:
df = pd.read_csv('../../data/openings.csv')

In [95]:
df.head()

,id,project_id,title,description,tags
0,d569ec70-f3fa-421c-88db-0773b26a2475,22a9acc5-428a-41d4-b056-9bebe9183afe,Designer,Need a good designer fast,"['figma', 'ui', 'ux']"
1,aa0d5847-b00b-42e0-8986-6185ecffd449,22a9acc5-428a-41d4-b056-9bebe9183afe,Content Writer,Some description,[]
2,c888d326-0c57-4ac5-8629-5ee397fd0f6c,22a9acc5-428a-41d4-b056-9bebe9183afe,Manager,management,[]
3,1067c159-be01-4f89-a1cf-a32cdd67bb72,6f6235eb-a751-43f2-8bad-506c884635db,Backend Developer,This is a new testing description,"['golang', 'javascript', 'django']"
4,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,6f6235eb-a751-43f2-8bad-506c884635db,Frontend Developer,need a good and fast paced frontend developer ...,"['react', 'next', 'ts']"


In [96]:
df['title']=df['title'].apply(lambda x:x.split())
df['description']=df['description'].apply(lambda x:x.split())
df['project_id']=df['project_id'].apply(lambda x:[x])

In [97]:
import ast

def parse(obj):
    try:
        obj = ast.literal_eval(obj)
        return obj
    except:
        return obj

df['tags']=df['tags'].apply(parse)

In [98]:
df['keys']=df['title']+df['description']+df['tags']+df['project_id']

In [99]:
df.head()

,id,project_id,title,description,tags,keys
0,d569ec70-f3fa-421c-88db-0773b26a2475,[22a9acc5-428a-41d4-b056-9bebe9183afe],[Designer],"[Need, a, good, designer, fast]","[figma, ui, ux]","[Designer, Need, a, good, designer, fast, figm..."
1,aa0d5847-b00b-42e0-8986-6185ecffd449,[22a9acc5-428a-41d4-b056-9bebe9183afe],"[Content, Writer]","[Some, description]",[],"[Content, Writer, Some, description, 22a9acc5-..."
2,c888d326-0c57-4ac5-8629-5ee397fd0f6c,[22a9acc5-428a-41d4-b056-9bebe9183afe],[Manager],[management],[],"[Manager, management, 22a9acc5-428a-41d4-b056-..."
3,1067c159-be01-4f89-a1cf-a32cdd67bb72,[6f6235eb-a751-43f2-8bad-506c884635db],"[Backend, Developer]","[This, is, a, new, testing, description]","[golang, javascript, django]","[Backend, Developer, This, is, a, new, testing..."
4,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,[6f6235eb-a751-43f2-8bad-506c884635db],"[Frontend, Developer]","[need, a, good, and, fast, paced, frontend, de...","[react, next, ts]","[Frontend, Developer, need, a, good, and, fast..."


In [88]:
import nltk
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prathammishra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [100]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
import string

ps=PorterStemmer()

custom_stopwords = ["need", "want", "this", "that", "fast"]

def stem(x):
    L = []
    tagged_tokens = pos_tag(x)
    for token, pos in tagged_tokens:
        token=token.lower()
        if pos != 'JJ' and pos != 'JJR' and pos != 'JJS' and token not in custom_stopwords:  # Remove adjectives
            stemmed_token = ps.stem(token)
            if stemmed_token not in L and stemmed_token not in stopwords.words("english") and stemmed_token not in string.punctuation:
                L.append(stemmed_token)
    return " ".join(L)

In [101]:
df.loc[:,'keys']=df['keys'].apply(stem)

In [102]:
df=df[['id','title','keys']]

In [103]:
df.tail()

,id,title,keys
0,d569ec70-f3fa-421c-88db-0773b26a2475,[Designer],design figma
1,aa0d5847-b00b-42e0-8986-6185ecffd449,"[Content, Writer]",content writer descript
2,c888d326-0c57-4ac5-8629-5ee397fd0f6c,[Manager],manag
3,1067c159-be01-4f89-a1cf-a32cdd67bb72,"[Backend, Developer]",backend develop test descript golang javascrip...
4,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,"[Frontend, Developer]",frontend develop dajkwdbjdbajwkdaw react ts


In [104]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000)

In [105]:
vectors = cv.fit_transform(df['keys']).toarray()

In [106]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectors)

In [107]:
similarities.shape

(5, 5)

In [110]:
df

,id,project_id,title,description,tags
0,d569ec70-f3fa-421c-88db-0773b26a2475,22a9acc5-428a-41d4-b056-9bebe9183afe,Designer,Need a good designer fast,"['figma', 'ui', 'ux']"
1,aa0d5847-b00b-42e0-8986-6185ecffd449,22a9acc5-428a-41d4-b056-9bebe9183afe,Content Writer,Some description,[]
2,c888d326-0c57-4ac5-8629-5ee397fd0f6c,22a9acc5-428a-41d4-b056-9bebe9183afe,Manager,management,[]
3,1067c159-be01-4f89-a1cf-a32cdd67bb72,6f6235eb-a751-43f2-8bad-506c884635db,Backend Developer,This is a new testing description,"['golang', 'javascript', 'django']"
4,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,6f6235eb-a751-43f2-8bad-506c884635db,Frontend Developer,need a good and fast paced frontend developer ...,"['react', 'next', 'ts']"


In [147]:
def recommend(opening_id):
    try:
        opening_index = df[df['id'].str.lower()==opening_id.lower()].index[0]
        distances = similarities[opening_index]
        opening_objs = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
        return [(df.iloc[i[0]].id, df.iloc[i[0]].title, f"{round(i[1]*100,2)}%") for i in opening_objs]
    except:
        return []

In [148]:
# for opening: Frontend Developer
recommend("2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b")

[('1067c159-be01-4f89-a1cf-a32cdd67bb72', 'Backend Developer', '16.9%'),
 ('d569ec70-f3fa-421c-88db-0773b26a2475', 'Designer', '0.0%'),
 ('aa0d5847-b00b-42e0-8986-6185ecffd449', 'Content Writer', '0.0%'),
 ('c888d326-0c57-4ac5-8629-5ee397fd0f6c', 'Manager', '0.0%')]

In [135]:
import pickle

with open('../../models/openings/similarities.pickle', 'wb') as f:
    pickle.dump(similarities, f)